In [1]:
%load_ext autoreload
%autoreload 2

# COP27 Comparaison de plusieurs sujet

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import datetime

import sys
sys.path.append("../")

%load_ext autoreload
%autoreload 2

from quotaclimat.utils.plotly_theme import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Update les data de la journée dans ../data/cop27/multi_keywords


# 2. Récupération des données

In [3]:
DATA_PATH = "../data/cop27/multi_keywords/"
os.listdir(DATA_PATH)


['20221106_20221030_routedurhum.xlsx',
 '20221106_20221030_cop27.xlsx',
 '20221106_20221106_veyrac.xlsx',
 '20221106_20221106_ukrainepoutine2.xlsx',
 '20221106_20221030_gregoirefournas.xlsx',
 '20221106_20221030_elonmusk.xlsx',
 '20221106_20221106_ukrainepoutine.xlsx',
 '20221106_20221106_midterm.xlsx']

In [7]:
from quotaclimat.data_processing.read_format_deduplicate import read_and_format_all_data_dump
from quotaclimat.data_analytics.exploration import filter_data_between_hours
from quotaclimat.data_analytics.exploration import show_mentions_by_channel
from quotaclimat.data_analytics.exploration import show_mentions_by_time_of_the_day
from quotaclimat.data_analytics.exploration import show_mentions_over_time
from quotaclimat.utils.plotly_theme import *


In [8]:
data = read_and_format_all_data_dump(path_folder=DATA_PATH, path_channel_metadata=None)
data.shape

(11119, 15)

## Restreindre perimètre d'analyse

In [9]:
top_audiences = pd.read_excel("../data/channels.xlsx",sheet_name = "top_audiences")
top_audiences["channel_id"] = top_audiences["channel_name"] + "_" + top_audiences["media"]
top_channels_tv = top_audiences.query("media=='TV'")["channel_name"].tolist()
top_channels_radio = top_audiences.query("media=='Radio'")["channel_name"].tolist()

data['is_top_audiance'] = False
data.loc[data.channel_name.isin(top_channels_tv + top_channels_radio), 'is_top_audiance'] = True

data_top_audiance = data.loc[data['is_top_audiance'], ]
data_at_pick_hours_6_00 = filter_data_between_hours(data_top_audiance, min_hour="06:00", max_hour="24:00")
data_at_pick_hours_6_10 = filter_data_between_hours(data_top_audiance, min_hour="06:00", max_hour="10:00")
data_at_pick_hours_19_21 = filter_data_between_hours(data_top_audiance, min_hour="19:00", max_hour="21:00")


# Figures

In [23]:
def get_percentage_per_day(df):
    minutes_per_sample = 2
    minutes_covered_per_day = ((max(df.date.dt.hour) - min(df.date.dt.hour)) + 1) * 60
    nb_days = (df.date.max() - df.date.min()).days + 1

    df_percentage = df.set_index('date').groupby(pd.Grouper(freq='D')).count()['index'] * minutes_per_sample / (minutes_covered_per_day * 25*df.media.nunique()) * 100
    df_percentage.name = 'percentage'
    return df_percentage



In [24]:
data_at_pick_hours_6_00.keyword.unique()

array(['cop27', 'elonmusk', 'ukrainepoutine', 'midterm',
       'ukrainepoutine2', 'gregoirefournas', 'veyrac', 'routedurhum'],
      dtype=object)

In [25]:
data_at_pick_hours_6_00_6nov_all = data_at_pick_hours_6_00[data_at_pick_hours_6_00.date.dt.day==6]
data_at_pick_hours_6_10_6nov_radio = data_at_pick_hours_6_10[(data_at_pick_hours_6_10.date.dt.day==6)&data_at_pick_hours_6_10.radio]
data_at_pick_hours_19_21_6nov_TV = data_at_pick_hours_19_21[(data_at_pick_hours_19_21.date.dt.day==6) & (~data_at_pick_hours_19_21.radio)]

In [26]:
df_matinal_jt = pd.concat([data_at_pick_hours_19_21_6nov_TV, data_at_pick_hours_6_10_6nov_radio])

In [27]:
percentage_per_keyword_6_00 = round((data_at_pick_hours_6_00_6nov_all.groupby('keyword').count() * 2 / (60 * 18 * 50) * 100),1)['index']
percentage_per_keyword_19_21 = round((data_at_pick_hours_19_21_6nov_TV.groupby('keyword').count() * 2 / (60 * 2 * 25) * 100),1)['index']
percentage_per_keyword_6_10 = round((data_at_pick_hours_6_10_6nov_radio.groupby('keyword').count() * 2 / (60 * 4 * 25) * 100),1)['index']


percentage_per_keyword_matinal_JT = round((df_matinal_jt.groupby('keyword').count() * 2 / (60 * 7 * 50) * 100),1)['index']


fig_ = go.Figure()
fig_.add_trace(go.Bar(y=percentage_per_keyword_6_10, x=percentage_per_keyword_6_10.index, name='Radio matinal entre 6h et 10h'))
fig_.add_trace(go.Bar(y=percentage_per_keyword_19_21, x=percentage_per_keyword_19_21.index, name='TV Journal entre 19h et 21h'))
fig_.add_trace(go.Bar(y=percentage_per_keyword_6_00, x=percentage_per_keyword_6_00.index, name='Radio + TV entre 6h et 00h'))
#fig_.add_trace(go.Bar(y=percentage_per_keyword_matinal_JT, x=percentage_per_keyword_matinal_JT.index, name='Matinal + JT'))

fig_.update_layout(title='% de represention des mots clef sur different plage horaire')
fig_.show()


In [28]:
pd.DataFrame([percentage_per_keyword_6_00, percentage_per_keyword_19_21, percentage_per_keyword_6_10] , index=['Radio + TV entre 6h et 00h', 'Journal TV entre 19h et 21h', 'Radio + TV entre 6h et 00h']).T

,Radio + TV entre 6h et 00h,Journal TV entre 19h et 21h,Radio + TV entre 6h et 00h
keyword,,,
cop27,2.2,2.3,3.9
elonmusk,0.2,0.4,0.3
gregoirefournas,0.1,0.9,0.1
midterm,1.7,4.9,1.8
routedurhum,0.8,0.2,1.2
ukrainepoutine,1.4,4.5,1.2
ukrainepoutine2,0.2,NaN,NaN
veyrac,0.2,0.1,0.1


In [29]:
data_to_use = data_at_pick_hours_6_00.copy()
data_percentage_elon = get_percentage_per_day(data_to_use[(data_to_use.keyword == 'elonmusk')])
data_percentage_cop27 = get_percentage_per_day(data_to_use[(data_to_use.keyword == 'cop27')])
data_percentage_routeduRhum = get_percentage_per_day(data_to_use[(data_to_use.keyword == 'routedurhum')])
data_percentage_gregoirefournas = get_percentage_per_day(data_to_use[(data_to_use.keyword == 'gregoirefournas')])
data_percentage_ukraine = get_percentage_per_day(data_to_use[(data_to_use.keyword == 'ukrainepoutine')])




In [37]:
data_to_use = data_at_pick_hours_6_00.copy()
data_percentage_elon = get_percentage_per_day(data_to_use[(data_to_use.keyword == 'elonmusk')])
data_percentage_cop27 = get_percentage_per_day(data_to_use[(data_to_use.keyword == 'cop27')])
data_percentage_routeduRhum = get_percentage_per_day(data_to_use[(data_to_use.keyword == 'routedurhum')])
data_percentage_gregoirefournas = get_percentage_per_day(data_to_use[(data_to_use.keyword == 'gregoirefournas')])
data_percentage_ukraine = get_percentage_per_day(data_to_use[(data_to_use.keyword == 'ukrainepoutine')])


fig = go.Figure()
fig.add_trace(go.Scatter(y=data_percentage_elon, x=data_percentage_elon.index, name='Elon Musk'))
fig.add_trace(go.Scatter(y=data_percentage_cop27, x=data_percentage_cop27.index, name='COP 27'))
fig.add_trace(go.Scatter(y=data_percentage_gregoirefournas, x=data_percentage_gregoirefournas.index, name='Gregoire Fournas'))
fig.add_trace(go.Scatter(y=data_percentage_routeduRhum, x=data_percentage_routeduRhum.index, name='route du Rhum'))
fig.add_trace(go.Scatter(y=data_percentage_ukraine, x=data_percentage_ukraine.index, name='Ukraine Poutine'))

fig.update_layout(title='Couverture en % des sujet d actualité TV + Radio de 6h à 00h')
fig.show()

In [31]:
data_to_use = data_at_pick_hours_6_10[data_at_pick_hours_6_10.channel_name.isin(top_channels_radio )].copy()
data_percentage_elon = get_percentage_per_day(data_to_use[(data_to_use.keyword == 'elonmusk')])
data_percentage_cop27 = get_percentage_per_day(data_to_use[(data_to_use.keyword == 'cop27')])
data_percentage_routeduRhum = get_percentage_per_day(data_to_use[(data_to_use.keyword == 'routedurhum')])
data_percentage_gregoirefournas = get_percentage_per_day(data_to_use[(data_to_use.keyword == 'gregoirefournas')])
data_percentage_ukraine = get_percentage_per_day(data_to_use[(data_to_use.keyword == 'ukrainepoutine')])


fig = go.Figure()
fig.add_trace(go.Scatter(y=data_percentage_elon, x=data_percentage_elon.index, name='Elon Musk'))
fig.add_trace(go.Scatter(y=data_percentage_cop27, x=data_percentage_cop27.index, name='COP 27'))
fig.add_trace(go.Scatter(y=data_percentage_gregoirefournas, x=data_percentage_gregoirefournas.index, name='Gregoire Fournas'))
fig.add_trace(go.Scatter(y=data_percentage_routeduRhum, x=data_percentage_routeduRhum.index, name='route du Rhum'))
fig.add_trace(go.Scatter(y=data_percentage_ukraine, x=data_percentage_ukraine.index, name='Ukraine Poutine'))

fig.update_layout(title='Couverture en % des sujet d actualité Radio de 6h à 10h')
fig.show()

In [33]:
data_to_use = data_at_pick_hours_19_21[data_at_pick_hours_19_21.channel_name.isin(top_channels_tv)].copy()

data_percentage_elon = get_percentage_per_day(data_to_use[(data_to_use.keyword == 'elonmusk')])
data_percentage_cop27 = get_percentage_per_day(data_to_use[(data_to_use.keyword == 'cop27')])
data_percentage_routeduRhum = get_percentage_per_day(data_to_use[(data_to_use.keyword == 'routedurhum')])
data_percentage_gregoirefournas = get_percentage_per_day(data_to_use[(data_to_use.keyword == 'gregoirefournas')])
data_percentage_ukraine = get_percentage_per_day(data_to_use[(data_to_use.keyword == 'ukrainepoutine')])

fig = go.Figure()
#fig.add_trace(go.Scatter(y=data_percentage_elon, x=data_percentage_elon.index, name='Elon Musk'))
fig.add_trace(go.Scatter(y=data_percentage_cop27, x=data_percentage_cop27.index, name='COP 27'))
#fig.add_trace(go.Scatter(y=data_percentage_gregoirefournas, x=data_percentage_gregoirefournas.index, name='Gregoire Fournas'))
#fig.add_trace(go.Scatter(y=data_percentage_routeduRhum, x=data_percentage_routeduRhum.index, name='route du Rhum'))
#fig.add_trace(go.Scatter(y=data_percentage_ukraine, x=data_percentage_ukraine.index, name='Ukraine Poutine'))

fig.update_layout(title='Couverture en % des sujet d actualité top TV de 19h à 21h')
fig.show()

# Autre sujets abordés